# Burseras and Friends
## How´s the environment ?
## Which are their friends ? 

In this example we will explore how to query and select a Node (Bursera, Node Type : Family) in the Tree Of Life.
We will extract the associated Cells and explore which kind of other taxa share that space. 
Let´s start!

![Burcera Environment](http://www.geographylists.com/mex_pv_forest.jpg "Bursera and its environment")


### Import modules libraries and load ploting style

In [1]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph

### Select the Node Bursera

Bursera is a Family of Plants. We want to select a Node in the Tree Of Life that is of the type Family and the name starts 
with "Burser". The expression "_.name=~STRING" means that we will look into the attribute "name" using a regular expression 
"=~" followed by any number of characters ".*"

In [2]:
bursera = list(Family.select(graph).where("_.name=~'Burser.*'")).pop()

The result is a list. Given that there is only one Node of the type Family that has the name "Burser" we can extract
the only element of the list using the standard method pop().

In [3]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

### Inspect taxonomy

In [4]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [5]:
bursera.getParent()

<TreeNode type: Order id = 933 name: Sapindales>

In [6]:
bursera.getParent().getParent().getParent().getParent()

<TreeNode type: Kingdom id = 6 name: Plantae>

The Tree Node is an object and has defined methods (refer to the documentation). One of this methods extract the cells in the spatial lattice that has associated Bursera occurrences. 

In [7]:
cells_with_burseras = bursera.cells

The result is a generator object. This is helpfull when the number of cells is big and surpasses the capacity of the machine
or the analyst.

In [8]:
cells_with_burseras

<generator object __iter__ at 0x7f13798dc9b0>

In this example we want to get only the first 10 cells.

In [9]:
cells_with_burseras = list(cells_with_burseras)

## *Oke Oke*!, 
Now we want to get the subtrees of Life constrainted by the geographical attribute of this cells.
For doing this, simply get the occurrences within each cell with the method *occurrencesHere()* and use this output as the constructor argument.

In [10]:
c = cells_with_burseras[0]

In [11]:
c.occurrencesHere()

[<Occurrence pk=540443>,
 <Occurrence pk=536897>,
 <Occurrence pk=538067>,
 <Occurrence pk=538060>,
 <Occurrence pk=552009>,
 <Occurrence pk=1686161>,
 <Occurrence pk=2198672>,
 <Occurrence pk=1900876>,
 <Occurrence pk=2153230>,
 <Occurrence pk=2144783>,
 <Occurrence pk=556012>,
 <Occurrence pk=553369>,
 <Occurrence pk=536899>,
 <Occurrence pk=1798523>]

## Random subselection of cells
The number of cells is very big. An option for processing is to select a random uniform set of indexes. We can achieve this with the following strategy.

1. make a random sample selection


### Let's create a random subset of 100 cells.

In [12]:
import numpy.random as rnd

In [13]:
n = 100

In [14]:
indices = rnd.randint(0,len(cells_with_burseras),n)

In [15]:
selected_cells = [ cells_with_burseras[i] for i in indices ]

### Get the subtrees within the selected cells

Let's create first a Constructor function

In [16]:
ToTree = lambda cell : TreeNeo(cell.occurrencesHere())

and map it to the selected cells

In [17]:
%time trees = map(ToTree,selected_cells)

CPU times: user 3min 40s, sys: 5.58 s, total: 3min 46s
Wall time: 5min 4s


#### Uff! it took 3 minutes, but it's a lot of data

In [18]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 1056- >,
 <LocalTree Of Life | Root: LUCA - n.count : 25- >,
 <LocalTree Of Life | Root: LUCA - n.count : 33- >,
 <LocalTree Of Life | Root: LUCA - n.count : 38- >,
 <LocalTree Of Life | Root: LUCA - n.count : 74- >,
 <LocalTree Of Life | Root: LUCA - n.count : 3- >,
 <LocalTree Of Life | Root: LUCA - n.count : 54- >,
 <LocalTree Of Life | Root: LUCA - n.count : 56- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1- >,
 <LocalTree Of Life | Root: LUCA - n.count : 307- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 37- >,
 <LocalTree Of Life | Root: LUCA - n.count : 376- >,
 <LocalTree Of Life | Root: LUCA - n.count : 245- >,
 <LocalTree Of Life | Root: LUCA - n.count : 9- >,
 <LocalTree Of Life | Root: LUCA - n.count : 27- >,
 <LocalTree Of Life | Root: LUCA - n.count : 438- >,
 <LocalTree Of Life | Root: LUCA - n.count : 621- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTre

#### Get the UNION of the these 100 trees~

In [19]:
big_tree = reduce(lambda a,b : a + b , trees)

INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging

In [20]:
big_tree.richness

29822

I told you, loads of data.
We can explore the content of this tree accessing to the attributes that start with the suffix ''to_''

In [21]:
mushrooms = big_tree.to_Fungi

Mushrooms is a TreeNode, i.e. not a complete TreeNeo object. To cast it (convert) to a TreeNode, and explore the families,classes, etc within it we must *plant-it* .

In [22]:
m = mushrooms.plantTreeNode()

In [23]:
type(mushrooms)

drivers.tree_builder.LocalTree

In [24]:
type(m)

drivers.tree_builder.TreeNeo

In [25]:
m.classes

[<TreeNode | Class: Agaricomycetes - n.count : 7- >,
 <TreeNode | Class: Lecanoromycetes - n.count : 18- >,
 <TreeNode | Class: Dothideomycetes - n.count : 2- >,
 <TreeNode | Class: Arthoniomycetes - n.count : 6- >,
 <TreeNode | Class: Pezizomycetes - n.count : 1- >,
 <TreeNode | Class: Sordariomycetes - n.count : 2- >]

In [26]:
big_tree.to_Protozoa

<TreeNode | Kingdom: Protozoa - n.count : 6- >

In [27]:
big_tree.to_Plantae

<TreeNode | Kingdom: Plantae - n.count : 8028- >

In [28]:
big_tree.to_Animalia

<TreeNode | Kingdom: Animalia - n.count : 21752- >

### Cnidaria in the "terrestrial" environment ?  

In [29]:
cnidaria = big_tree.to_Animalia.to_Cnidaria

Where is this ?

In [37]:
cnidaria_cells = cnidaria.getExactCells()

In [38]:
cnidaria_cells

[<Cell id=223271>, <Cell id=222137>]

In [39]:
c1 , c2 = cnidaria_cells

In [41]:
c1.cell

u'MULTIPOLYGON (((-110.2690634159987 24.18688980100021, -110.2200634159987 24.18688980100021, -110.2200634159987 24.13788980100021, -110.2690634159987 24.13788980100021, -110.2690634159987 24.18688980100021)))'

![Cnidaria cell 1](cnidaria_cell1.png)

In [43]:
# c2.cell
## Dont do it, name to big.

![Cnidaria cell 2](cnidaria_cell2.png)

Close to the sea, but a little bit off the shore.

In [ ]:
## 

In [46]:
beatle = big_tree.to_Animalia.to_Arthropoda.to_Insecta.to_Coleoptera

## How many of these have beatles ?

In [48]:
with_beatles = filter(lambda tree : tree.hasNode(beatle),trees)

In [49]:
len(with_beatles)

8

# Environmental Exploration
Remember that the big tree is defined in the geographical space given by the selected cells, i.e. the random sample.
We can get the environmental summary statistics of these cells with the following command.

In [54]:
bursera_env = big_tree.associatedData.getEnvironmentalVariablesCells(with_std=True)

In [55]:
bursera_env

{'MaxTemperature_mean': 14.238538843721749,
 'MaxTemperature_std': 3.958130561502632,
 'MeanTemperature_mean': 21.855606368563667,
 'MeanTemperature_std': 3.1927214613012853,
 'MinTemperature_mean': 29.470980126467968,
 'MinTemperature_std': 2.936706588522792,
 'Precipitation_mean': 61.102218834688294,
 'Precipitation_std': 39.941882359382795,
 'SolarRadiation_mean': 18363.57164634147,
 'SolarRadiation_std': 1053.2515593672322,
 'Vapor_mean': 1.6425022583559166,
 'Vapor_std': 0.5581388290085026,
 'WindSpeed_mean': 2.312020099367661,
 'WindSpeed_std': 0.6463689624565792}

### The same for point data

In [30]:
only_bursera = big_tree.to_Plantae.to_Magnoliophyta.to_Magnoliopsida.to_Sapindales.to_Burseraceae

In [31]:
pnts_env_burs = only_bursera.associatedData.getEnvironmentalVariablesPoints()

In [32]:
pnts_env_burs

,MaxTemperature_mean,MaxTemperature_std,MeanTemperature_mean,MeanTemperature_std,MinTemperature_mean,MinTemperature_std,Precipitation_mean,Precipitation_std,Vapor_mean,Vapor_std,SolarRadiation_mean,SolarRadiation_std,WindSpeed_mean,WindSpeed_std
0,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000
1,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
2,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
3,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
4,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
5,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
6,15.250000,4.044029,22.666667,3.836955,30.083333,3.639101,14.916667,17.665684,1.916667,0.640095,18959.250000,3270.691877,3.250000,0.433013
7,14.500000,5.751811,20.666667,5.749396,27.000000,5.522681,14.083333,11.324445,1.500000,0.645497,18710.333333,4306.416614,2.833333,0.372678
8,17.000000,5.700877,24.416667,5.765679,31.416667,5.765679,18.500000,20.434856,1.666667,0.849837,18372.083333,4116.958089,2.666667,0.471405
9,17.000000,5.700877,24.416667,5.765679,31.416667,5.765679,18.500000,20.434856,1.666667,0.849837,18372.083333,4116.958089,2.666667,0.471405


In [42]:
nodes_iters = map(lambda oc : (oc.pullbackRasterNodes('MaxTemperature'),oc),only_bursera.occurrences)

In [33]:
chiquibum = [list(node) for (node,x) in nodes_iters]

In [34]:
chiquibum

[[<Raster Data type: MaxTemp-30s value = 22.0 >],
 [<Raster Data type: MaxTemp-30s value = 9.0 >],
 [<Raster Data type: MaxTemp-30s value = 15.0 >],
 [<Raster Data type: MaxTemp-30s value = 17.0 >],
 [<Raster Data type: MaxTemp-30s value = 14.0 >],
 [<Raster Data type: MaxTemp-30s value = 22.0 >],
 [<Raster Data type: MaxTemp-30s value = 14.0 >],
 [<Raster Data type: MaxTemp-30s value = 10.0 >],
 [<Raster Data type: MaxTemp-30s value = 10.0 >],
 [<Raster Data type: MaxTemp-30s value = 15.0 >],
 [<Raster Data type: MaxTemp-30s value = 9.0 >],
 [<Raster Data type: MaxTemp-30s value = 9.0 >],
 [<Raster Data type: MaxTemp-30s value = 9.0 >],
 [<Raster Data type: MaxTemp-30s value = 14.0 >],
 [<Raster Data type: MaxTemp-30s value = 14.0 >],
 [<Raster Data type: MaxTemp-30s value = 10.0 >],
 [<Raster Data type: MaxTemp-30s value = 9.0 >],
 [<Raster Data type: MaxTemp-30s value = 8.0 >],
 [<Raster Data type: MaxTemp-30s value = 23.0 >],
 [<Raster Data type: MaxTemp-30s value = 23.0 >],
 [<Ras

In [44]:
nodes = map(lambda node_occ : (list(node_occ[0])[0],node_occ[1]),nodes_iters)

IndexError: list index out of range

In [45]:
n0 = nodes_iters[0]

In [46]:
n0

(<generator object __iter__ at 0x7f276b506500>, <Occurrence pk=1909672>)

In [47]:
list(n0[0])

[]

In [80]:
test_f = lambda node_occ : (list(node_occ[0])[0],node_occ[1])

In [49]:
only_bursera_data_points_env = only_bursera.pullbackRasterNodes('MaxTemperature')

In [50]:
only_bursera_data_points_env

[([<Raster Data type: MaxTemp-30s value = 22.0 >], <Occurrence pk=1909672>),
 ([<Raster Data type: MaxTemp-30s value = 9.0 >], <Occurrence pk=1918758>),
 ([<Raster Data type: MaxTemp-30s value = 15.0 >], <Occurrence pk=1553064>),
 ([<Raster Data type: MaxTemp-30s value = 17.0 >], <Occurrence pk=1802769>),
 ([<Raster Data type: MaxTemp-30s value = 14.0 >], <Occurrence pk=1909682>),
 ([<Raster Data type: MaxTemp-30s value = 22.0 >], <Occurrence pk=1553067>),
 ([<Raster Data type: MaxTemp-30s value = 14.0 >], <Occurrence pk=1553081>),
 ([<Raster Data type: MaxTemp-30s value = 10.0 >], <Occurrence pk=1659710>),
 ([<Raster Data type: MaxTemp-30s value = 10.0 >], <Occurrence pk=1928915>),
 ([<Raster Data type: MaxTemp-30s value = 15.0 >], <Occurrence pk=1909668>),
 ([<Raster Data type: MaxTemp-30s value = 9.0 >], <Occurrence pk=1659737>),
 ([<Raster Data type: MaxTemp-30s value = 9.0 >], <Occurrence pk=1928945>),
 ([<Raster Data type: MaxTemp-30s value = 9.0 >], <Occurrence pk=3162747>),
 ([

In [52]:
only_bursera_data_points_env = only_bursera.associatedData.getValuesFromPoints??

In [ ]:
only_bursera_data_points_env = only_bursera.associatedData.getValuesFromPoints

In [ ]:
only_bursera_data_points_env = only_bursera.associatedData.getValuesFromPoints

In [ ]:
only_bursera_data_points_env = only_bursera.associatedData.getEnvironmentalVariablesPoints

## General aproach

In [52]:
type(big_tree.levels)

list

In [45]:
birds = t.to_Animalia.to_Chordata.to_Aves

In [46]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [47]:
birds

<TreeNode | Class: Aves - n.count : 8- >

In [48]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 4- >,
 <LocalTree Of Life | Root: LUCA - n.count : 37- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 5- >,
 <LocalTree Of Life | Root: LUCA - n.count : 9- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 59- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [49]:
filter(lambda tree : tree.hasNode(birds),trees)

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [56]:
getNeighbourhood = lambda tree : tree.getNeighboringTrees()